# Binomial Option Pricing Model
*Homework for the Introduction to Quantitative Finance Course*

Implementation of Cox, Ross and Rubinstein (CRR) Method to price European Options and Binomial Tree Model pricing American Options using numpy arrays to increase the computing efficiency.

In [111]:
import numpy as np
from functools import wraps
from time import time

### Generic timing wrapper function
We will use this to benchmark the two binomial models

In [112]:
def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print(f'{f.__name__} took: {te-ts:2.6f} sec')
        return result
    return wrap

## Binomial Tree Representation
Stock tree can be represented using nodes (i,j) and intial stock price $S_0$

$S_{i,j} = S_0 u^{j} d^{i-j}$

where:
- $i$ = time step (0 to N)
- $j$ = number of up movements (0 to i)
- $i-j$ = number of down movements

$C_{i,j}$ represents contract price at each node (i,j). Where $C_{N,j}$ represents final payoff function that we can define. 

### European Option Characteristics
- **Call Option:** $C_{N,j} = \max(S_{N,j}-K, 0)$
- **Put Option:** $C_{N,j} = \max(K-S_{N,j}, 0)$

### American Option Characteristics

For an American Put Option:

**At maturity ($i=N$):**
$$C_{N,j} = \max(K - S_{N,j}, 0) = (K - S_{N,j})^+$$

**At earlier nodes ($i < N$):**

Calculate the **continuation value** (holding the option):
$$V_{\text{cont}} = e^{-r\Delta t} [p \cdot C_{i+1,j+1} + (1-p) \cdot C_{i+1,j}]$$

Calculate the **intrinsic value** (early exercise):
$$V_{\text{intrinsic}} = (K - S_{i,j})^+$$

**Option value is the maximum:**
$$C_{i,j} = \max(V_{\text{intrinsic}}, V_{\text{cont}})$$

where:
- $p$ is the risk-neutral probability (constant): $p = \frac{e^{r\Delta t} - d}{u - d}$
- $C_{i+1,j+1}$ is the option value after an up move
- $C_{i+1,j}$ is the option value after a down move

For American Call: replace $(K-S_{i,j})^+$ with $(S_{i,j}-K)^+$

## Initial parameters

In [113]:
S0 = 40       # initial stock price
K = 45        # strike price
T = 1      # time to maturity in years
r = 0.03        # annual risk-free rate
N = 1         # number of time steps
sigma = 0.25    # Annualised stock price volatility
opttype = 'P'  # Option Type 'C' or 'P'
exctype = 'A'  # Exercise Type 'E' or 'A'

### Tree statistics
Returns theoretical number of price paths and actual number of nodes evaluated in a recombining binomial tree.

In [114]:
def tree_statistics(N):
    paths = 2 ** N
    nodes = (N + 1) * (N + 2) // 2
    return paths, nodes

### CRR Tree Parameters
$$u = e^{\sigma\sqrt{\Delta t}}, \quad d = \frac{1}{u} = e^{-\sigma\sqrt{\Delta t}}$$
$$p = \frac{e^{r\Delta t} - d}{u - d}, \quad \text{disc} = e^{-r\Delta t}$$

In [115]:
# European CRR Binomial Tree Method
@timing
def CRR_method(K,T,S0,r,N,sigma,opttype='C'):
    #precomute constants
    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u

    # no-arbitrage condition
    if not (d < np.exp(r*dt) < u):
        raise ValueError(
            f"No-arbitrage condition violated! "
            f"Must have d < e^(r*dt) < u. "
            f"Got: d={d:.6f}, e^(r*dt)={np.exp(r*dt):.6f}, u={u:.6f}"
        )
    
    p = (np.exp(r*dt) - d) / (u-d)
    disc = np.exp(-r*dt)

    # initialise asset prices at maturity - Time step N
    S = np.zeros(N+1)
    S[0] = S0*d**N
    for j in range(1,N+1):
        S[j] = S[j-1]*u/d

    # initialise option values at maturity
    if opttype == 'C':
        C = np.maximum(0, S - K)
    else:
        C = np.maximum(0, K - S)

    # step backwards through tree
    for i in range(N - 1,-1,-1):
        C = disc * (p*C[1:] + (1-p)*C[:-1])

    paths, nodes = tree_statistics(N)

    return C[0], paths, nodes


# American Binomial Tree Method (Fast Implementation)
@timing
def american_fast_tree(K,T,S0,r,N,sigma,opttype='P'):
    #precompute values
    dt = T/N
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u

    # no-arbitrage condition
    if not (d < np.exp(r*dt) < u):
        raise ValueError(
            f"No-arbitrage condition violated! "
            f"Must have d < e^(r*dt) < u. "
            f"Got: d={d:.6f}, e^(r*dt)={np.exp(r*dt):.6f}, u={u:.6f}"
        )
    
    p = (np.exp(r*dt) - d) / (u-d)
    disc = np.exp(-r*dt)

    # initialise stock prices at maturity
    S = S0 * (d**np.arange(N,-1,-1)) * (u**np.arange(0,N+1,1))

    # option payoff
    if opttype == 'P':
        C = np.maximum(0, K - S)
    else:
        C = np.maximum(0, S - K)

    # backward recursion through the tree
    for i in np.arange(N-1,-1,-1):
        C_cont = disc * (p*C[1:] + (1 - p)*C[:-1])
        
        # stock prices at time step i
        S = S0 * (d**np.arange(i,-1,-1)) * (u**np.arange(0,i+1,1))

        # intrinsic value for american option
        if opttype == 'P':
            C_intrinsic = np.maximum(0, K - S)
        else:
            C_intrinsic = np.maximum(0, S - K)

        # max(continuation, intrinsic)
        C = np.maximum(C_cont, C_intrinsic)

    paths, nodes = tree_statistics(N)

    return C[0], paths, nodes

In [116]:
print("\n--- Valuation ---")
print(f"Parameters: S0={S0}, K={K}, T={T}, r={r}, N={N}")
print(f"Option: {'Call' if opttype=='C' else 'Put'}")
print(f"Exercise: {'European' if exctype=='E' else 'American'}\n")

if exctype == 'E':
    price, paths, nodes = CRR_method(K, T, S0, r, N, sigma, opttype)
    model = "European Binomial Tree"
else:
    price, paths, nodes = american_fast_tree(K, T, S0, r, N, sigma, opttype)
    model = "American Binomial Tree"

print(f"Model: {model}")
print(f"Volatility: {sigma:.2%}")
print(f"Option value: {price:.6f}")
print(f"Theoretical number of price paths: {paths}")
print(f"Number of nodes actually evaluated: {nodes}")



--- Valuation ---
Parameters: S0=40, K=45, T=1, r=0.03, N=1
Option: Put
Exercise: American

american_fast_tree took: 0.000149 sec
Model: American Binomial Tree
Volatility: 25.00%
Option value: 6.744847
Theoretical number of price paths: 2
Number of nodes actually evaluated: 3
